In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://books.toscrape.com/catalogue/page-{}.html"
books = []

for page in range(1, 51):  # 50 pages
    url = base_url.format(page)
    res = requests.get(url)
    if res.status_code != 200:
        break
    soup = BeautifulSoup(res.text, "html.parser")
    items = soup.find_all("article", class_="product_pod")

    for item in items:
        title = item.h3.a["title"]
        price = item.find("p", class_="price_color").text.strip()
        availability = item.find("p", class_="instock availability").text.strip()
        star_class = item.p["class"][1]  # e.g. "Three"
        books.append([title, price, availability, star_class])

df_books = pd.DataFrame(books, columns=["Title", "Price", "Availability", "Star Rating"])
df_books.to_csv("books.csv", index=False)
df_books.head()


,Title,Price,Availability,Star Rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five


In [2]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium pandas


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,272 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,297 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jamm

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
driver.get("https://www.imdb.com/chart/top/")
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

movies = []
rows = soup.select("li.ipc-metadata-list-summary-item")

for rank, row in enumerate(rows, start=1):
    title_tag = row.select_one("h3")
    title = title_tag.text.replace(str(rank)+".", "").strip()
    year = row.select_one("span.ipc-title__subtext").text.strip("()")
    rating = row.select_one("span.ipc-rating-star--rating").text
    movies.append([rank, title, year, rating])

df_imdb = pd.DataFrame(movies, columns=["Rank", "Title", "Year", "Rating"])
df_imdb.to_csv("imdb_top250.csv", index=False)
df_imdb.head()


,Rank,Title,Year,Rating


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.timeanddate.com/weather/"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

cities = []
rows = soup.select("table tbody tr")

for row in rows:
    city_tag = row.find("a")
    if not city_tag:
        continue
    city = city_tag.text
    temp = row.find_all("td")[1].text.strip()
    condition = row.find_all("td")[2].text.strip()
    cities.append([city, temp, condition])

df_weather = pd.DataFrame(cities, columns=["City", "Temperature", "Condition"])
df_weather.to_csv("weather.csv", index=False)
df_weather.head()


,City,Temperature,Condition
